In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
import datetime
current_date = datetime.datetime.now().date()
llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/Users/ybaciu/Library/Python/3.12/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
print(vectordb._collection.count())

38


In [13]:
question = "Who were the airstrikes against?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [14]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

### RetrievalQA chain

In [15]:
from langchain.chains import RetrievalQA

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [17]:
result = qa_chain({"query": question})

In [18]:
result["result"]

'The airstrikes were conducted against Iranian-backed militias in Iraq and Syria, specifically targeting the Islamic Revolutionary Guards Corps (IRGC) Quds Force and affiliated militia groups. The strikes targeted facilities such as command and control operations, intelligence centers, rockets and missiles, unmanned aerial vehicle storages, and logistics and munition supply chain facilities.'

### Prompt


In [19]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [20]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [21]:
question = "Are green bell peppers an ingredient in the Cheesy Green Chile Bean Bake recipe?"

In [22]:
result = qa_chain({"query": question})

In [23]:
result["result"]

'Yes, green bell peppers are an ingredient in the Cheesy Green Chile Bean Bake recipe. Thanks for asking!'

In [24]:
question = "Is turmeric an ingredient in the Cheesy Green Chile Bean Bake recipe?"

In [25]:
result = qa_chain({"query": question})

In [26]:
result["result"]

"I don't know. Thanks for asking!"